In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
import os
warnings.filterwarnings('ignore')
os.chdir("../../")

In [3]:
import time

In [4]:
from juggling_smt import *

In [5]:
import modele
import ipywidgets
import pythreejs

colors = ["blue", "red", "green", "yellow", "purple", "cyan", "magenta"]
sides = [-1, 1, 1]

# Musique à jouer

In [6]:
# Au clair de la lune
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [6]:
music = [(1, "re5"), (2, "do5"), (3, "re5"), (4, "do5"),
         (5, "re5"), (7, "do5"), (9, "mi5"),
         (11, "re5"), (12, "fa5"), (13, "la5"), (14, "re6"), (15, "fa6")]

In [7]:
# Frères jacques
music = [( 1, "fa4"), ( 2, "sol4"), ( 3, "la4"), ( 4, "fa4"),
         ( 5, "fa4"), ( 6, "sol4"), ( 7, "la4"), ( 8, "fa4"),
         ( 9, "la4"), (10, "la#4"), (11, "do5"),
         (13, "la4"), (14, "la#4"), (15, "do5")]

In [8]:
# In the hall of the moutain king
music = [( 1, "la3"), ( 2, "si3"), ( 3, "do4"), ( 4, "re4"),
         ( 5, "mi4"), ( 6, "do4"), ( 7, "mi4"),
         ( 9, "re#4"), (10, "si3"), (11, "re#4"),
         (13, "re4"), (14, "la#3"), (15, "re4")]

# Contraintes

In [13]:
nb_hands = 2
max_height = 5
max_weight = 3
forbidden_multiplex = [(1, 2), (1, 3), (1, 4), (2, )]

# Algorithme

In [8]:
balls, throws = music_to_throws(music)

In [43]:
T = throws_to_define_fun(throws)

In [45]:
t, b, hmax = z3.Ints('t b hmax')

In [ ]:
T()

In [75]:
solve_with_smt_solver(balls, throws, nb_hands, max_height, max_weight, forbidden_multiplex)

sat
<class 'z3.z3.ModelRef'>
Pair(0, 3)
Pair(1, 0)
Pair(15, 2)


In [69]:
print(solver)

[ForAll([t, b, hmax, m, h],
        Implies(And(m >= 0,
                    m < 2,
                    h >= 0,
                    h <= hmax,
                    affect(t, b, hmax) == Pair(m, h)),
                If(Or(And(t == 0, b == "do", hmax == 1),
                      And(t == 0, b == "re", hmax == 4),
                      And(t == 0, b == "mi", hmax == 5),
                      And(t == 1, b == "do", hmax == 1),
                      And(t == 2, b == "do", hmax == 1),
                      And(t == 3, b == "do", hmax == 6),
                      And(t == 4, b == "re", hmax == 3),
                      And(t == 5, b == "mi", hmax == 5),
                      And(t == 7, b == "re", hmax == 4),
                      And(t == 9, b == "do", hmax == 4),
                      And(t == 11, b == "re", hmax == 1)),
                   True,
                   False))),
 ForAll([t, b, hmax],
        Implies(If(Or(And(t == 0, b == "do", hmax == 1),
                      And(t == 0, b == "r

In [70]:
solver.check()

sat

In [71]:
m = solver.model()

In [11]:
ec_instance = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                             forbidden_multiplex, multiple_throws)

## Résolution avec Exact Cover

In [9]:
sol1 = get_solution_with_dlx(ec_instance)

In [10]:
len(sol1.rows)

11

In [11]:
print_juggling(sol1)

{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : do -- 1 --> 0
{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : re -- 1 --> 1
      {'mi'}          {'re', 'do'}    : mi -- 1 --> 1
        {}         {'re', 'do', 'mi'} : 
        {}         {'re', 'do', 'mi'} : re -- 1 --> 0
      {'re'}          {'do', 'mi'}    : 
      {'re'}          {'do', 'mi'}    : do -- 1 --> 0
   {'re', 'do'}          {'mi'}       : mi -- 1 --> ?
   {'re', 'do'}            {}         : re -- 1 --> 1
      {'do'}             {'re'}       : re -- 1 --> ?
      {'do'}               {}         : do -- 1 --> ?


In [12]:
balls1, pattern1 = juggling_sol_to_simulator(sol1, colors)

sides1 = [-1, 1, 1]

model1 = modele.Model(balls1, pattern1)
slider1 = ipywidgets.FloatSlider(min=0, max=40, step=0.05)
view1 = modele.View(model1, sides1)
slider1.observe(lambda change: view1.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view1.widget,slider1])

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

## Résolution avec MILP

In [13]:
sol2 = solve_exact_cover_with_milp(ec_instance, True)

In [14]:
len(sol2.rows)

11

In [15]:
print_juggling(sol2)

{'re', 'do', 'mi'}      {}      : do -- 1 --> 1
   {'re', 'mi'}       {'do'}    : re -- 3 --> 1
                                  mi -- 4 --> 1
                                  do -- 1 --> 0
      {'do'}            {}      : do -- 1 --> 1
        {}            {'do'}    : 
        {}         {'re', 'do'} : re -- 3 --> 0
        {}         {'do', 'mi'} : 
        {}         {'do', 'mi'} : do -- 3 --> 0
      {'re'}          {'mi'}    : mi -- 3 --> ?
      {'re'}            {}      : re -- 3 --> 1
      {'do'}            {}      : 
      {'do'}            {}      : do -- 3 --> ?
        {}            {'re'}    : re -- 1 --> ?
        {}              {}      : 


In [16]:
balls2, pattern2 = juggling_sol_to_simulator(sol2, colors)

sides2 = [-1, 1, 1]

model2 = modele.Model(balls2, pattern2)
slider2 = ipywidgets.FloatSlider(min=0, max=40, step=0.05)
view2 = modele.View(model2, sides2)
slider2.observe(lambda change: view2.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view2.widget,slider2])

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

## Fonctions tout en un

In [7]:
solve_and_simulate(music, nb_hands, max_height, max_weight, forbidden_multiplex, colors, sides, method="DLX")

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

In [18]:
solve_and_simulate(music, nb_hands, max_height, max_weight, forbidden_multiplex, colors, sides, method="MILP")

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

JSProxyWidget(status='deferring flush until render')

In [19]:
solve_and_print(music, nb_hands, max_height, max_weight, forbidden_multiplex, method="DLX")

{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : do -- 1 --> 0
{'re', 'do', 'mi'}         {}         : do -- 1 --> 1
   {'re', 'mi'}          {'do'}       : re -- 1 --> 1
      {'mi'}          {'re', 'do'}    : mi -- 1 --> 1
        {}         {'re', 'do', 'mi'} : 
        {}         {'re', 'do', 'mi'} : re -- 1 --> 0
      {'re'}          {'do', 'mi'}    : 
      {'re'}          {'do', 'mi'}    : do -- 1 --> 0
   {'re', 'do'}          {'mi'}       : mi -- 1 --> ?
   {'re', 'do'}            {}         : re -- 1 --> 1
      {'do'}             {'re'}       : re -- 1 --> ?
      {'do'}               {}         : do -- 1 --> ?


In [20]:
solve_and_print(music, nb_hands, max_height, max_weight, forbidden_multiplex, method="MILP")

{'re', 'do', 'mi'}      {}      : do -- 1 --> 1
   {'re', 'mi'}       {'do'}    : re -- 3 --> 1
                                  mi -- 4 --> 1
                                  do -- 1 --> 0
      {'do'}            {}      : do -- 1 --> 1
        {}            {'do'}    : 
        {}         {'re', 'do'} : re -- 3 --> 0
        {}         {'do', 'mi'} : 
        {}         {'do', 'mi'} : do -- 3 --> 0
      {'re'}          {'mi'}    : mi -- 3 --> ?
      {'re'}            {}      : re -- 3 --> 1
      {'do'}            {}      : 
      {'do'}            {}      : do -- 3 --> ?
        {}            {'re'}    : re -- 1 --> ?
        {}              {}      : 
